## Comparison between the iterative Image Space Restoration Algorithm (ISRA) and the Richardson-Lucy Algorithm (RLA) 
Generated Voigt functions



In [ ]:
#required libraries
%matplotlib qt
import numpy as np
import hyperspy.api as hs
from ncempy.io import dm
import matplotlib.pyplot as plt
from scipy.signal import peak_widths, find_peaks
import math

Functions

In [ ]:
#convert text files
def txtconverter(numpy_array):
    file = str(numpy_array).replace('[','')
    file = file.replace(']','')
    data = np.fromstring(file, sep=',')
    return data

def find_counts(data):
    counts=data[1:-1:2]
    return counts

def find_ev(data):
    ev=data[0:-1:2]
    return ev
    
def hyperspy_plot(ev, counts):
    s = hs.signals.EELSSpectrum(counts)
    s.axes_manager[0].scale = np.diff(ev).mean()
    s.axes_manager[0].unit = 'eV'
    s.axes_manager[0].offset = ev[0]
    s.axes_manager[0].name = 'Energy'
    return s

#Normalize spectrum based on total electron counts
def normalizeS(S):
	S_norm = S/np.sum(S)
	return S_norm

#ISRA algorithm
def ISRA(iterations, PSF, Spec):
    ISRA5 = np.copy(Spec)
    ISRA1 = np.convolve(PSF, Spec, mode='same')
    for i in range(iterations):
        ISRA2 = np.convolve(PSF, ISRA5, mode='same')
        ISRA3 = np.convolve(PSF, ISRA2, mode='same')
        ISRA4 = np.divide(ISRA1, ISRA3)
        ISRA5 = np.multiply(ISRA4, ISRA5)
    return ISRA5

#Richardson-Lucy algorithm (code from Edson Bellido)
def RL(iterations, PSF, Spec):
    RL4 = np.copy(Spec)
    for i in range(iterations):
        RL1 = np.convolve(PSF, RL4, mode='same')
        RL2 = np.divide(Spec,RL1)
        RL3 = np.convolve(PSF, RL2, mode='same')
        RL4 = np.multiply(RL3, RL4)
    return RL4

Load files

In [ ]:
#load file as numpy array
Signal = np.loadtxt("D:\Downloads\Signal1.txt",dtype="str")
PSF = np.loadtxt("D:\Downloads\PSF1.txt", dtype='str')

#convert text file to usable numpy array
signal = txtconverter(Signal)
psf = txtconverter(PSF)

#separate data into counts and ev
psf_counts = find_counts(psf)
signal_counts = find_counts(signal)
signal_ev = find_ev(signal)

Deconvolve

In [ ]:
ISRA_deconvolve = ISRA(10, psf_counts, signal_counts)
a = hyperspy_plot(signal_ev, ISRA_deconvolve)
a.plot()

In [ ]:
RL_deconvolve = RL(10, psf_counts, signal_counts)
s = hyperspy_plot(signal_ev, RL_deconvolve)
s.plot()